### Preprocess

In [1]:
from sklearn.neighbors import DistanceMetric
from math import radians
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse
import torch
from torch.utils.dlpack import to_dlpack, from_dlpack
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_undirected, is_undirected, convert
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx
#from torch_geometric.nn import GCNConv
import torch.nn.functional as F
pd.options.mode.chained_assignment = None
from dgl import DGLGraph
import time

def MAPELoss(output, target):
    return torch.mean(torch.abs((target - output) / target))  

Using backend: pytorch


In [2]:
cities_df = pd.DataFrame({
    'house':['h1','h2','h3','h4','h5','h6','t1','t2','t3'],
    'lat':[12.9716,19.076,28.7041,22.5726,13.0827,23.2599,12.9713,19.003,28.7048],
    'lon':[77.5946,72.877,77.1025,80.639,80.2707,77.4126,77.5920,72.8605,77.2000],
    'x1':[20,35,24,33,35,18,19,32,23],
    'x2':[5,5,7,13,16,21,6,6,6],
    'y':[1200,1500,2000,1780,1450,3000,1300,1600,1800],
    'train_mask':[True,True,True,True,True,True,False,False,False],
    'val_mask':[False,False,False,False,False,False,True,True,False],
    'test_mask':[False,False,False,False,False,False,False,False,True]})
cities_df

,house,lat,lon,x1,x2,y,train_mask,val_mask,test_mask
0,h1,12.9716,77.5946,20,5,1200,True,False,False
1,h2,19.0760,72.8770,35,5,1500,True,False,False
2,h3,28.7041,77.1025,24,7,2000,True,False,False
3,h4,22.5726,80.6390,33,13,1780,True,False,False
4,h5,13.0827,80.2707,35,16,1450,True,False,False
5,h6,23.2599,77.4126,18,21,3000,True,False,False
6,t1,12.9713,77.5920,19,6,1300,False,True,False
7,t2,19.0030,72.8605,32,6,1600,False,True,False
8,t3,28.7048,77.2000,23,6,1800,False,False,True


In [3]:
def geo_transform(df, m):
    df['lat'] = np.radians(df['lat'])
    df['lon'] = np.radians(df['lon'])
    
    dist = DistanceMetric.get_metric('haversine')
    
    df_dist = pd.DataFrame(dist.pairwise(df[['lat','lon']].to_numpy())*6373,  columns=df.house.unique(), index=df.house.unique())
    
    D = df_dist.values
    D[D < m] = 1
    D[D >= m] = 0
    
    G = nx.from_numpy_matrix(D)
    edge_index = from_networkx(G).edge_index
    
    x = torch.tensor(df[['x1','x2']].values, dtype=torch.float)
    y = torch.tensor(df[['y']].values, dtype=torch.float)
    train_mask = torch.BoolTensor(df[['train_mask']].values)
    val_mask = torch.BoolTensor(df[['val_mask']].values)
    test_mask = torch.BoolTensor(df[['test_mask']].values)
    
    data = Data(x=x, y=y, edge_index=edge_index)
    
    g = DGLGraph(nx.from_numpy_matrix(D))
    features = x
    labels = y
    
    return edge_index, g, features, labels, train_mask, val_mask, test_mask  

In [4]:
edge_index, g, features, labels, train_mask, val_mask, test_mask   = geo_transform(cities_df, 800)

/Users/shao/anaconda3/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [5]:
edge_index

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6,
         6, 6, 7, 7, 7, 8, 8, 8, 8],
        [0, 4, 6, 1, 5, 7, 2, 3, 5, 8, 2, 3, 5, 8, 0, 4, 6, 1, 2, 3, 5, 7, 8, 0,
         4, 6, 1, 5, 7, 2, 3, 5, 8]])

In [6]:
g

Graph(num_nodes=9, num_edges=33,
      ndata_schemes={}
      edata_schemes={})

### Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.fc.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)

        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        self.z = edges.src['z']
        self.e = edges.data['e']
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        self.alpha = alpha
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        self.h = h
        return {'h': h}

    def forward(self, h):
        # equation (1)
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2)
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return torch.cat(head_outs, dim=1)
        else:
            # merge using average
            return torch.mean(torch.stack(head_outs))
        
class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        # Be aware that the input dimension is hidden_dim*num_heads since
        # multiple head outputs are concatenated together. Also, only
        # one attention head in the output layer.
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        return h

In [8]:
# import time
# import numpy as np

# # create the model, 2 heads, each head has hidden size 8
# net = GAT(g,
#           in_dim=features.size()[1],
#           hidden_dim=8,
#           out_dim=1,
#           num_heads=2)

# # create optimizer
# optimizer = torch.optim.Adam(net.parameters(), lr=1e-1)

# # main loop
# dur = []
# for epoch in range(10):
#     if epoch >= 3:
#         t0 = time.time()

#     logits = net(features)
#     loss = MAPELoss(logits[mask], labels[mask])

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
#         epoch, loss.item(), np.mean(dur)))

In [9]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def step(self, loss, model):
        score = loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score > self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0
        return self.early_stop

    def save_checkpoint(self, model):
        '''Saves model when validation loss decrease.'''
        torch.save(model.state_dict(), 'es_checkpoint.pt')



# create the model, 2 heads, each head has hidden size 8
net = GAT(g,
          in_dim=features.size()[1],
          hidden_dim=8,
          out_dim=1,
          num_heads=2)        
        
print(net)
stopper = EarlyStopping(patience=100)

# create optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=1e-1)     

        
        
        
# initialize graph
dur = []
for epoch in range(10):
    net.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    logits = net(features)
    loss = MAPELoss(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    train_loss = MAPELoss(logits[train_mask], labels[train_mask])

    val_loss = MAPELoss(logits[val_mask], labels[val_mask])
    if stopper.step(val_loss, net):
        break

    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | TrainLoss {:.4f} |"
          " ValLoss {:.4f} | ETputs(KTEPS) {:.2f}".
          format(epoch, np.mean(dur), loss.item(), train_loss,
                 val_loss, g.num_edges() / np.mean(dur) / 1000))

GAT(
  (layer1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=2, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
      (1): GATLayer(
        (fc): Linear(in_features=2, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
    )
  )
  (layer2): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=16, out_features=1, bias=False)
        (attn_fc): Linear(in_features=2, out_features=1, bias=False)
      )
    )
  )
)
Epoch 00000 | Time(s) nan | Loss 0.9951 | TrainLoss 0.9951 | ValLoss 0.9946 | ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 0.9841 | TrainLoss 0.9841 | ValLoss 0.9807 | ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 0.9724 | TrainLoss 0.9724 | ValLoss 0.9670 | ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.0076 | Loss 0.9594 | TrainLoss 0.9594 | ValLoss 0.9517 | ETp

/Users/shao/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/shao/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
from scipy.sparse import lil_matrix
from sklearn.preprocessing import normalize

def preprocess_attention(edge_atten, g, to_normalize=True):
    """Organize attentions in the form of csr sparse adjacency
    matrices from attention on edges. 

    Parameters
    ----------
    edge_atten : numpy.array of shape (# edges, # heads, 1)
        Un-normalized attention on edges.
    g : dgl.DGLGraph.
    to_normalize : bool
        Whether to normalize attention values over incoming
        edges for each node.
    """
    n_nodes = g.number_of_nodes()
    num_heads = edge_atten.shape[1]
    all_head_A = [lil_matrix((n_nodes, n_nodes)) for _ in range(num_heads)]
    for i in range(n_nodes):
        predecessors = list(g.predecessors(i))
        edges_id = g.edge_ids(predecessors, i)
        for j in range(num_heads):
            all_head_A[j][i, predecessors] = edge_atten[edges_id, j].data.cpu().numpy()
            #all_head_A[j][i, predecessors] = edge_atten[edges_id, j, 0].data.cpu().numpy()
    if to_normalize:
        for j in range(num_heads):
            all_head_A[j] = normalize(all_head_A[j], norm='l1').tocsr()
    return all_head_A

In [11]:
A = g.edata['e']     
A = preprocess_attention(A, g) 

In [12]:
print(A[0])

  (0, 0)	0.3333333333333333
  (0, 4)	0.3333333333333333
  (0, 6)	0.3333333333333333
  (1, 1)	0.3221031918534302
  (1, 5)	0.35579361629313955
  (1, 7)	0.3221031918534302
  (2, 2)	0.24952355068327076
  (2, 3)	0.24952355068327076
  (2, 5)	0.25142934795018773
  (2, 8)	0.24952355068327076
  (3, 2)	0.24952355068327076
  (3, 3)	0.24952355068327076
  (3, 5)	0.25142934795018773
  (3, 8)	0.24952355068327076
  (4, 0)	0.3333333333333333
  (4, 4)	0.3333333333333333
  (4, 6)	0.3333333333333333
  (5, 1)	0.15654216128312695
  (5, 2)	0.17140194723605384
  (5, 3)	0.17140194723605384
  (5, 5)	0.17270983572558452
  (5, 7)	0.15654216128312695
  (5, 8)	0.17140194723605384
  (6, 0)	0.3333333333333333
  (6, 4)	0.3333333333333333
  (6, 6)	0.3333333333333333
  (7, 1)	0.3221031918534302
  (7, 5)	0.35579361629313955
  (7, 7)	0.3221031918534302
  (8, 2)	0.24952355068327076
  (8, 3)	0.24952355068327076
  (8, 5)	0.25142934795018773
  (8, 8)	0.24952355068327076


In [13]:
edge_index

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6,
         6, 6, 7, 7, 7, 8, 8, 8, 8],
        [0, 4, 6, 1, 5, 7, 2, 3, 5, 8, 2, 3, 5, 8, 0, 4, 6, 1, 2, 3, 5, 7, 8, 0,
         4, 6, 1, 5, 7, 2, 3, 5, 8]])